In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [3]:
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)
train_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
6118,51,Private,39264,Some-college,10,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,>50K
23204,58,Private,51662,10th,6,Married-civ-spouse,Other-service,Wife,White,Female,0,0,8,United-States,<=50K
29590,40,Private,326310,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,44,United-States,<=50K
18116,37,Private,222450,HS-grad,9,Never-married,Sales,Not-in-family,White,Male,0,2339,40,El-Salvador,<=50K
33964,62,Private,109190,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,15024,0,40,United-States,>50K


In [4]:
label = 'class'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count        500
unique         2
top        <=50K
freq         365
Name: class, dtype: object


In [5]:
save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train_data)

Beginning AutoGluon training ...
AutoGluon will save models to "agModels-predictClass/"
AutoGluon Version:  0.2.0
Train Data Rows:    500
Train Data Columns: 14
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' >50K', ' <=50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify positive_class in Predictor init.
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    15695.73 MB
	Train

In [6]:
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv | Columns = 15 / 15 | Rows = 9769 -> 9769


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,31,Private,169085,11th,7,Married-civ-spouse,Sales,Wife,White,Female,0,0,20,United-States
1,17,Self-emp-not-inc,226203,12th,8,Never-married,Sales,Own-child,White,Male,0,0,45,United-States
2,47,Private,54260,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1887,60,United-States
3,21,Private,176262,Some-college,10,Never-married,Exec-managerial,Own-child,White,Female,0,0,30,United-States
4,17,Private,241185,12th,8,Never-married,Prof-specialty,Own-child,White,Male,0,0,20,United-States


In [7]:
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.8397993653393387
Evaluations on test data:
{
    "accuracy": 0.8397993653393387,
    "balanced_accuracy": 0.7437076677780596,
    "mcc": 0.5295565206264157,
    "f1": 0.6242496998799519,
    "precision": 0.7038440714672441,
    "recall": 0.5608283002588438
}


Predictions:  
 0        <=50K
1        <=50K
2         >50K
3        <=50K
4        <=50K
         ...  
9764     <=50K
9765     <=50K
9766     <=50K
9767     <=50K
9768     <=50K
Name: class, Length: 9769, dtype: object


In [8]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost,0.842666,0.85,0.086939,0.006937,1.412479,0.086939,0.006937,1.412479,1,True,11
1,RandomForestGini,0.841335,0.84,0.216864,0.107666,1.428998,0.216864,0.107666,1.428998,1,True,5
2,RandomForestEntr,0.840721,0.83,0.216390,0.108107,0.516354,0.216390,0.108107,0.516354,1,True,6
3,LightGBM,0.839799,0.85,0.030590,0.014302,0.175851,0.030590,0.014302,0.175851,1,True,4
4,WeightedEnsemble_L2,0.839799,0.85,0.032908,0.014934,0.542035,0.002317,0.000632,0.366184,2,True,14
5,LightGBMXT,0.839390,0.83,0.022714,0.013082,7.024005,0.022714,0.013082,7.024005,1,True,3
6,CatBoost,0.837957,0.84,0.021558,0.010429,6.088101,0.021558,0.010429,6.088101,1,True,7
7,ExtraTreesEntr,0.834783,0.82,0.217278,0.107739,0.516962,0.217278,0.107739,0.516962,1,True,9
8,ExtraTreesGini,0.834476,0.82,0.219182,0.107780,0.518595,0.219182,0.107780,0.518595,1,True,8
9,LightGBMLarge,0.827823,0.83,0.031156,0.013556,0.325446,0.031156,0.013556,0.325446,1,True,13


# Maximizing predictive performance¶

In [9]:
time_limit = 60  # for quick demonstration only, you should set this to longest time you are willing to wait (in seconds)
metric = 'roc_auc'  # specify your evaluation metric here
predictor = TabularPredictor(label, eval_metric=metric).fit(train_data, time_limit=time_limit, presets='best_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20210503_055525/"
Presets specified: ['best_quality']
Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "AutogluonModels/ag-20210503_055525/"
AutoGluon Version:  0.2.0
Train Data Rows:    500
Train Data Columns: 14
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' >50K', ' <=50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify positive_class in Predictor init.
Us

In [10]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_BAG_L1,0.902783,0.887489,0.080645,0.045495,3.839947,0.080645,0.045495,3.839947,1,True,7
1,LightGBMXT_BAG_L1,0.900161,0.881380,0.221364,0.060853,0.930427,0.221364,0.060853,0.930427,1,True,3
2,WeightedEnsemble_L2,0.897023,0.900233,7.773673,1.218794,48.529014,0.005292,0.001246,1.322062,2,True,14
3,LightGBM_BAG_L1,0.892347,0.866991,0.119764,0.060637,0.910145,0.119764,0.060637,0.910145,1,True,4
4,XGBoost_BAG_L1,0.891681,0.866575,0.430661,0.029503,0.909221,0.430661,0.029503,0.909221,1,True,11
5,RandomForestEntr_BAG_L1,0.888119,0.886301,0.216969,0.094713,0.517905,0.216969,0.094713,0.517905,1,True,6
6,RandomForestGini_BAG_L1,0.886598,0.884698,0.217026,0.095014,0.517517,0.217026,0.095014,0.517517,1,True,5
7,ExtraTreesGini_BAG_L1,0.881065,0.892927,0.218217,0.099178,0.518237,0.218217,0.099178,0.518237,1,True,8
8,ExtraTreesEntr_BAG_L1,0.880851,0.893912,0.219856,0.096843,0.517950,0.219856,0.096843,0.517950,1,True,9
9,NeuralNetFastAI_BAG_L1,0.878074,0.864191,0.915170,0.139274,8.309110,0.915170,0.139274,8.309110,1,True,10


# Regression (predicting numeric table columns)

In [11]:
age_column = 'age'
print("Summary of age variable: \n", train_data[age_column].describe())

Summary of age variable: 
 count    500.00000
mean      39.65200
std       13.52393
min       17.00000
25%       29.00000
50%       38.00000
75%       49.00000
max       85.00000
Name: age, dtype: float64


In [12]:
predictor_age = TabularPredictor(label=age_column, path="agModels-predictAge").fit(train_data, time_limit=60)
performance = predictor_age.evaluate(test_data)

Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "agModels-predictAge/"
AutoGluon Version:  0.2.0
Train Data Rows:    500
Train Data Columns: 14
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == int and many unique label-values observed).
	Label info (max, min, mean, stddev): (85, 17, 39.652, 13.52393)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    15451.07 MB
	Train Data (Original)  Memory Usage: 0.32 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...

In [13]:
predictor_age.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-10.538595,-11.313580,1.867007,0.376075,6.823546,0.004941,0.000652,0.506928,2,True,12
1,ExtraTreesMSE,-10.691115,-11.480752,0.219697,0.108433,0.513192,0.219697,0.108433,0.513192,1,True,7
2,RandomForestMSE,-10.746518,-11.602848,0.219566,0.108350,0.511767,0.219566,0.108350,0.511767,1,True,5
3,LightGBMXT,-10.753344,-11.814712,0.065118,0.014002,0.213778,0.065118,0.014002,0.213778,1,True,3
4,CatBoost,-10.800412,-11.744795,0.022241,0.010553,0.431703,0.022241,0.010553,0.431703,1,True,6
5,LightGBM,-10.972156,-11.929546,0.037690,0.014294,0.192942,0.037690,0.014294,0.192942,1,True,4
6,XGBoost,-11.121008,-12.174270,0.102236,0.007074,0.248006,0.102236,0.007074,0.248006,1,True,9
7,LightGBMLarge,-11.598649,-12.167606,0.080257,0.015089,0.420255,0.080257,0.015089,0.420255,1,True,11
8,NeuralNetMXNet,-13.234982,-13.377319,1.195520,0.112718,4.205230,1.195520,0.112718,4.205230,1,True,10
9,KNeighborsUnif,-14.902058,-15.686937,0.103416,0.102777,0.002591,0.103416,0.102777,0.002591,1,True,1
